## Comment:

I used functions from seminars (with some modificaions regarding inputs/outputs). Also I dedicated one cell for all imports and one cell for all functions, so then I could present outputs in more convenient (at least I believe so :) ) form.

### 0.0 ) Make imports and define functions (from seminars with Yash)

In [139]:
#Defining Libraries:

import hashlib
import base58
import os
import hashlib
import ecdsa
import binascii
import bitcoin
from ecdsa import SigningKey, SECP256k1
from prettytable import PrettyTable

In [140]:
# Function for printing metadata in readable format
def print_metadata(metadata):
    for idx, data in enumerate(metadata):
        print('Data entry #', idx + 1)
        print('''\t\tPrivate Key: {}\n
               Public Key: {}\n
               Compressed_Key: {}\n
               Main Address: {}\n
               Testnet Address: {}\n'''.format(data[0],
                                             data[1],
                                             data[2],
                                             data[3],
                                             data[4]))

    print()
# Tesnet address generation and basic functions (hashing, etc)

class variableInteger():    
    def reverse_string_in_pair(a_str_ip):
        i = 0
        rev_a = []
        while i != len(a_str_ip):
            #print(a[i] + a[i+1])
            temp_value = a_str_ip[i] + a_str_ip[i+1]
            rev_a.append(temp_value)
            i = i + 2
        rev_a.reverse()
        return ''.join(rev_a)

    def padding(b):
        if int(len(b) % 2) == 0:
            return b
        else:
            return '0' + b

    def variable_integer(a):
        
        temp = int(len(a)/2)
            
        if temp <= int('0xfc',16):
            return hex(temp)[2:]
        elif temp > int('0xfc',16) and temp <= int('0xffff',16):
            return 'fd' +  reverse_string_in_pair(padding(hex(temp)[2:]))
        elif temp > int('0xffff',16) and temp <= int('0xffffffff',16):
            return 'fe' + reverse_string_in_pair(padding(hex(temp)[2:]))
        elif temp > int('0xffffffff',16) and temp <= int('0xffffffffffffffff',16):
            return 'ff' + reverse_string_in_pair(padding(hex(temp)[2:]))

        
#Defining Function For Hashing: 1. Sha256 and 2. Ripemd160
def hashing(a):
    first_sha256 = hashlib.sha256(a)
    ripemd160 = hashlib.new("ripemd160")
    ripemd160.update(first_sha256.digest())
    return_0 = ripemd160.digest()
    return_1 = bytes.fromhex("00") + ripemd160.digest()
    return_2 = bytes.fromhex("6f") + ripemd160.digest()
    return return_0, return_1, return_2

#Function for Checksum
def checksum(b):
    checksum_full = hashlib.sha256(hashlib.sha256(b).digest()).digest()
    new_checksum = checksum_full[:4]
    return b + new_checksum

#Function to convert to base58    
def to_base58(c):
    return base58.b58encode(c).decode('utf-8')

def compressed_key(d):
    a = d.hex()

    if a[-1] == '0' or a[-1] == '2' or a[-1] == '4' or a[-1] == '6' or a[-1] == '8' or a[-1] == 'a' or a[-1] == 'c' or a[-1] == 'e':
        return bytes.fromhex("02") + d
    
    else:
        return bytes.fromhex("03") + d
    
def generate_address(print_output = True):
    
    #generating Random number of 32 bytes
    private_key = os.urandom(32).hex()
    private_key_from_hex = bytes.fromhex(private_key)

    #Keeping same private key, Generating Public Key
    import ecdsa # pip install ecdsa
    sk = ecdsa.SigningKey.from_string(private_key_from_hex, curve = ecdsa.SECP256k1) 


    #actual publickey to verify messages that singed using own pvt key
    verification_key = sk.verifying_key
    #making Public Key:
    public_key = bytes.fromhex("04") + verification_key.to_string()
    compressed_public_key = compressed_key(verification_key.to_string())
    compressed_public_key_hex = compressed_public_key.hex()[:66]
    #Converting public key to address:
    decoded_pubkey, mainnet_pubkey, testnet_pubkey = hashing(public_key)

    #DECODED PUBLIC KEY WILL BE REQUIRED
    #checksum:
    checksum_main_pubkey = checksum(mainnet_pubkey)
    checksum_test_pubkey = checksum(testnet_pubkey)


    main_address = to_base58(checksum_main_pubkey)
    test_address = to_base58(checksum_test_pubkey)


    if print_output:
        x = PrettyTable()
        x.field_names = ["Discription", "Value"]
        x.add_row(["Private Key, hex", private_key])
        x.add_row(["Public Key, hex", public_key.hex()])
        x.add_row(["Compressed Public Key, hex", compressed_public_key_hex])
        x.add_row(["MainNet Address, base58", main_address])
        x.add_row(["Testnet Address, base58", test_address])
        print(x)
    return(private_key, public_key.hex(), compressed_public_key_hex, main_address, test_address)




# Multisig generation 

def public_to_test(a):
    decoded_address = base58.b58decode(scriptaddress)
    address_1 = decoded_address[1:-4]
    hex_address = bytes.fromhex("C4") + address_1
    checksum_test_multsig = checksum(hex_address)
    test_address = to_base58(checksum_test_multsig)
    return test_address


# Double hash

def doublehash256(fname, lname): 
    str_ = fname + lname
    temp_hash = hashlib.sha256(str_.encode('utf-8')).digest()
    hash2 = hashlib.sha256(temp_hash).digest()
    return str(binascii.hexlify(hash2),"ascii")



# Generate Transaction

def doublehash256_txid(a): 
    temp_bin = binascii.unhexlify(a)
    temp_hash = hashlib.sha256(temp_bin).digest()
    hash2 = hashlib.sha256(temp_hash).digest()
    return str(binascii.hexlify(hash2),"ascii")

def bytewise_reverse(a):
    i = 0
    rev_a = []
    while i != len(a):
        temp_value = a[i] + a[i+1]
        rev_a.append(temp_value)
        i = i + 2
    rev_a.reverse()
    return ''.join(rev_a)

def bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = bytewise_reverse(str(temp_1).zfill(16))
    return temp_2

def tx_inputs_tx_id():
    previous_tx_hash = input("Enter Reference TxId: ")
    previous_tx_hash_reversed = bytewise_reverse(previous_tx_hash)
    previous_output_index = input("Enter Previous output index: ")
    previous_output_index = bytewise_reverse('{:08x}'.format(int(previous_output_index)))
    return previous_tx_hash_reversed + previous_output_index # + place_holder() +  'ffffffff'

def place_holder():
    previous_place_holder = input('''Enter Output Script 
    OP_HASH160(public_key)=RIPEMD160(SHA256(public_key)): ''')
    size_receiver_1 = hex(int(len(previous_place_holder)/2))[2:]
    script_Pubkey_prev_op = '76' + 'a9' + size_receiver_1 + previous_place_holder + '88' + 'ac'
    script_len_1 = hex(int(len(script_Pubkey_prev_op)/2))[2:]
    return script_len_1 + script_Pubkey_prev_op

def bytes_padding(temp_no):
    if len(temp_no)/2 != 0:
        return '0' + temp_no
    else:
        return temp_no

def tx_outputs(i):
    receiver = input("Enter Receiver's Address: ")
    value = input("Enter Value in BTC (0.0) to send:")
    value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
    receiver_address = base58.b58decode(receiver).hex()[2:-8]
    size_receiver = hex(int(len(receiver_address)/2))[2:]
    locking_Script = '76' + 'a9' + size_receiver + receiver_address + '88' + 'ac'
    script_len_2 = hex(int(len(locking_Script)/2))[2:]
    return value_in_hex_NYB + script_len_2 + locking_Script
    
    
def op_return(i):
    value_return = '0'*16
    data = input("Enter Data to send: ")
    text_from_bytes = binascii.b2a_hex(data.encode("utf-8")).decode()
    len_data = hex(int(len(text_from_bytes)/2))[2:]
    partial_data =  '6a'+ bytes_padding(len_data) + text_from_bytes
    return value_return + bytes_padding(hex(int(len(partial_data)/2))[2:]) + partial_data
    
def pushdata(len_r,len_s):
    if len_r == '20' and len_s == '20':
        return str(47)    
    elif len_r == '20' or len_s == '21':
        return str(48)
    elif len_r == '21' or len_s == '20':
        return str(48)
    else:
        return str(49)
    
def pubkey_opcode(a):
    return hex(int(int(len(a))/2))[2:]
    
def generate_raw_transaction(print_output = True):
    #Creating Raw Transaction for Double Hash
    #P2PKH
    version = 1
    version = bytewise_reverse('{:08d}'.format(int(version)))

    #_________________TALKING PREVIOUS TRANSACTIONS_________________
    utxo_ip = '01'
    prev_tx_datas = utxo_ip + tx_inputs_tx_id()


    #___________________________________________________________________
    utxo_output = input("Enter number of outputs, for example, 2: ")
    tx_output = ''
    for i in range(0, int(utxo_output)):
            tx_output = tx_outputs(i) + tx_output
    next_output_data = '0' + utxo_output + tx_output
    #____________________________________________________________________________

    locktime = '00000000'
    SigHashCode = '01000000'


    #Creating Raw transaction:
    print(
    """
    Raw_Transaction for 1 input Data and {} Output
    """.format(tx_output))
    #Printing Input PreSignature:
    
    raw_transaction = version + prev_tx_datas + next_output_data + locktime + SigHashCode
    if print_output:
        print(f'''
        Version {version}
        Previous data: {prev_tx_datas}
        Next output data: {next_output_data} 
        Locktime: {locktime} 
        SigHashCode: {SigHashCode}
        ''')
        print(raw_transaction)
    return(raw_transaction)    


# Functions that were used for sending bitcoins

def digital_signature(raw_tx):
    unsigned = raw_tx
    
    temp_bin = binascii.unhexlify(unsigned)
    temp_hash = hashlib.sha256(temp_bin).digest()
    tx_hash = hashlib.sha256(temp_hash).digest()
    private_key = input("Enter Private Key: ")
    signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)   
    temp_sig = str(binascii.hexlify(SIG),'ascii')
    return temp_sig     

def redeem_place_holder():
    redeemScript = input("Enter redeem Script: ")
    return variableInteger.variable_integer(redeemScript) + redeemScript

def reverse_string_in_pair(a_str_ip):
    i = 0
    rev_a = []
    while i != len(a_str_ip):
        temp_value = a_str_ip[i] + a_str_ip[i+1]
        rev_a.append(temp_value)
        i = i + 2
    rev_a.reverse()
    return ''.join(rev_a)

def create_new_transaction_3(tx_id, publickey, private_key, n_out, n_data_out):

    version_number = '01000000'
    number_inputs = '01'
    previous_transaction = tx_id
    previous_transaction = reverse(previous_transaction)
    
    previous_output = '0'
    output_script = '0' + previous_output + '000000'
    previous_PKH = input("Enter previous PKH: ")
    script = '1976a914' + previous_PKH + '88ac'
    sequence = 'ffffffff'
    tx_ip = version_number + number_inputs + previous_transaction + output_script + script + sequence


    #User_output:
    Outputs = str(n_out)
    #utxo outputs
    utxo_output = str(1) # the same all time -> hardcoded
    tx_output = ''
    for i in range(0, int(utxo_output)):
            tx_output = tx_outputs(i) + tx_output
    #data output
    data_output = str(n_data_out)
    op_ret = ''
    for i in range(0, int(data_output)):
            op_ret = op_return(i) + op_ret
    out_tx = '0' + Outputs + tx_output + op_ret + '0'*8 + '01000000'

    #Final Transaction:
    raw_transaction = tx_ip + out_tx
    print('''Raw transaction: ''', raw_transaction)

    #DIGITAL SIGNATURE
    unsigned = raw_transaction

    temp_bin = binascii.unhexlify(unsigned)
    temp_hash = hashlib.sha256(temp_bin).digest()
    hash2 = hashlib.sha256(temp_hash).hexdigest()
    tx_hash = hashlib.sha256(temp_hash).digest()

    signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)

    print("\n\nDigital Signature: ", str(binascii.hexlify(SIG),'ascii'))

    #Generating_SigScript:
    DER_input =  str(binascii.hexlify(SIG),'ascii')
    header = DER_input[0:2]
    Sig_Length = DER_input[2:4]
    r_integer = DER_input[4:6]
    r_length = DER_input[6:8]
    r = DER_input[8:8 + 2 * int(r_length,16)]
    s_integer = DER_input[8 + 2 * int(r_length,16): 10 + 2 * int(r_length,16)]
    s_length = DER_input[10 + 2 * int(r_length,16): 12 + 2 * int(r_length,16)]
    s = DER_input[12 + 2 * int(r_length,16):]

    pushdata_opcode = pushdata(r_length, s_length)
    sighash_code = '01'
 
    pubkey_pushdata_opcode = pubkey_opcode(publickey)

    final_SigScript = pushdata_opcode + header + Sig_Length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + pubkey_pushdata_opcode + publickey
    SigScript = pubkey_opcode(final_SigScript) + final_SigScript
    print("\nSigScript(hex): ",SigScript,"\n\n\n")

    #Broadcasting_Transaction:
    broadcasting_tx = version_number + number_inputs + previous_transaction + output_script + SigScript + sequence + out_tx [:-8]
    print('''Transaction to broadcast: ''', broadcasting_tx)


#----------------------- from here all is related to point 5
class variableInteger5:
    def reverse_string_in_pair(a_str_ip):
        i = 0
        rev_a = []
        while i != len(a_str_ip):
            #print(a[i] + a[i+1])
            temp_value = a_str_ip[i] + a_str_ip[i+1]
            rev_a.append(temp_value)
            i = i + 2
        rev_a.reverse()
        return ''.join(rev_a)
    
    def padding(b):
        if int(len(b) % 2) == 0:
            return b
        else:
            return '0' + b
    
    def variable_integer(a):
        if type(a) == int:
            temp = a
        else:
            temp = int(len(a)/2)
        
        if temp <= int('0xfc',16):
            return variableInteger5.padding(hex(temp)[2:])
        elif temp > int('0xfc',16) and temp <= int('0xffff',16):
            return 'fd' +  variableInteger5.reverse_string_in_pair(variableInteger5.padding(hex(temp)[2:]))
        elif temp > int('0xffff',16) and temp <= int('0xffffffff',16):
            return 'fe' + variableInteger5.reverse_string_in_pair(variableInteger5.padding(hex(temp)[2:]))
        elif temp > int('0xffffffff',16) and temp <= int('0xffffffffffffffff',16):
            return 'ff' + variableInteger5.reverse_string_in_pair(variableInteger5.padding(hex(temp)[2:]))

def tx_outputs_5(c):
    receiver = input("Enter Receiver's Address {}: ".format(c+1))
    if receiver[0] == 'm' or receiver[0] == 'n': 
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = '76' + 'a9' + variableInteger5.variable_integer(receiver_address) + receiver_address + '88' + 'ac'
        script_len_2 = variableInteger5.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script
    
    elif receiver[0] == '2':
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = 'a9' + variableInteger5.variable_integer(receiver_address) + receiver_address + '87'
        script_len_2 = variableInteger5.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script    
    
    
def create_new_transaction_5():

    sig_loop = str(2) # hardcoded
    #Making Raw_Transaction: 
    #input Basic
    version = '01000000'

    #input_multi_Signature:
    input_ms = '01' + tx_inputs_tx_id()
    redeem_Script = redeem_place_holder()
    tx_input_value = input_ms + redeem_Script + 'f'*8

    #output
    utxo_output = str(1) # hardcoded
    tx_output = ''
    for i in range(0, int(utxo_output)):
            tx_output = tx_outputs_5(i) + tx_output
    next_output_data =  variableInteger5.variable_integer(int(utxo_output)) + tx_output

    locktime = '00000000'
    SigHashCode = '01000000'

    tx_to_sign = version + tx_input_value + next_output_data + locktime + SigHashCode

    signatures = ''

    for i in range(0,int(sig_loop)):
        signature_temp = digital_signature(tx_to_sign) + '01'
        signature_temp_2 = variableInteger5.variable_integer(signature_temp) + signature_temp
        signatures = signatures + signature_temp_2

    SigScript_places = '00' + signatures + '4c' + redeem_Script 
    SigScript =  variableInteger5.variable_integer(SigScript_places) + SigScript_places

    Broadcasting_transaction = version + input_ms + SigScript + 'f'*8 + next_output_data + locktime
    print('''Transaction to broadcast: ''', Broadcasting_transaction)

### 0.1) Testnet address generetation

In [141]:
address_data_1 = generate_address(print_output = False)
address_data_2 = generate_address(print_output = False)
address_data_3 = generate_address(print_output = False)

address_1 = address_data_1[-1]
address_2 = address_data_2[-1]
address_3 = address_data_3[-1]

address_data_list = [address_data_1, address_data_2, address_data_3]
print('Testnet adresses:\n')
print()
print('Test address_1  : ', address_1)
print('Test address_2  : ', address_2)
print('Test address_3  : ', address_3)

print()
print('-----------------------------------')
print()
print('Metadata:')
print_metadata(address_data_list)

Testnet adresses:


Test address_1  :  mq7HfkJ4A2W6kBPD4ih5fFQtkGoD1mcpCF
Test address_2  :  muXBqnup9A3xScqLohymaX7AxZYz1oAvEB
Test address_3  :  mxVPWVJapiv5pGZ4PPgccTpMcS7G9NsyAy

-----------------------------------

Metadata:
Data entry # 1
		Private Key: 83c818cd2b883d38cec5a41d73fff8b5de1369310666391d51f6b187ba3dc388

               Public Key: 0447cec9869c73bfbb0d827abd4aca31580ea8ac485664584af6d3fc457f8e7022b44424b5a62111c14372152f4c8457b96a5e2180760d614f09d1d71acde2e664

               Compressed_Key: 0247cec9869c73bfbb0d827abd4aca31580ea8ac485664584af6d3fc457f8e7022

               Main Address: 1AbLNhD5M14qy4ubM9ihqLCZtHCW6dHQi2

               Testnet Address: mq7HfkJ4A2W6kBPD4ih5fFQtkGoD1mcpCF

Data entry # 2
		Private Key: c151c9ff4f01681cc9c4aa6a8c83e38b1dfbe00ec8dd3d992b3bb9f6cba02513

               Public Key: 04148a66d28626a869c6b51f777296420871d33f3fb5dafb9ca2cf595183f4d5c2891e64346ee3361768f5af0fcb800d2c8492fb92c1e47e30928ff82fda021a8b

               Compressed_Ke

### 0.2) MultiSignature address generation

In [142]:
m = 3 # Max
n = 2 # Min

pub_keys = []
for i in range(0,m):
    temp = address_data_list[i][1].format(i+1,m)

    pub_keys.append(temp)

In [143]:
   
rawscript = bitcoin.mk_multisig_script(pub_keys,n)
descript = bitcoin.deserialize_script(rawscript)
scriptaddress = bitcoin.scriptaddr(rawscript)
print("MultiSig Address: ", scriptaddress)

testnet_address = public_to_test(scriptaddress)
print("MultiSig Testnet Address: ", testnet_address) # this is what we need

print("\nRedeem Script:", rawscript)

MultiSig Address:  3QPHFmvE9rLXb33sgnptxXktbP6zPc8fU9
MultiSig Testnet Address:  2NFwVKWrFmJqsnpgRMvSmaUk9ojKAGbtv9C

Redeem Script: 52410447cec9869c73bfbb0d827abd4aca31580ea8ac485664584af6d3fc457f8e7022b44424b5a62111c14372152f4c8457b96a5e2180760d614f09d1d71acde2e6644104148a66d28626a869c6b51f777296420871d33f3fb5dafb9ca2cf595183f4d5c2891e64346ee3361768f5af0fcb800d2c8492fb92c1e47e30928ff82fda021a8b410476b89fcb368275796690e993cacac3e9be4df22ffbf19d67b7b1fda5fbff187720a8e2167abdb66daa91ffae074240d73335982ec11307c165c5e163b5702b3353ae


### 1) (1 point) Calculate double Hash_256 of your name. Take two inputs from the user, i.e., first name and last name. Concatenate string and take SHA256(SHA256(first name_lastname))

In [144]:
fname = 'Alexander'
lname = 'Shumilov'

print("Double Hash_256 of '%s' is : " % (fname + lname), doublehash256(fname, lname))

Double Hash_256 of 'AlexanderShumilov' is :  bec15355352ef7685aeb80d02944e5865c702df5db0c1405a1928cd75c96a48a


### 2) (1 point) Get some testnet Bitcoins to your testnet address_1 from any source. (Best source: https://coinfaucet.eu/en/btc-testnet/) . Provide the Transaction_ID. 

In [146]:
tx_id_2 = '518c585e870a897c26af6b8a29e2c2a8dbd774071d29bfbd4c28ca0a689da01b' 
pkh_2 = '6937728711d918890eeebbbb3819be859f821fe8' # to enter below

# source: https://testnet-faucet.mempool.co/

print('Test address_1 (from): ', address_1) # to enter below
print('Test address_2 (to): ', address_2) # to enter below
print('Previous PKH: ', pkh_2)
print()
print('Transaction ID: ', tx_id_2)

Test address_1 (from):  mq7HfkJ4A2W6kBPD4ih5fFQtkGoD1mcpCF
Test address_2 (to):  muXBqnup9A3xScqLohymaX7AxZYz1oAvEB
Previous PKH:  6937728711d918890eeebbbb3819be859f821fe8

Transaction ID:  518c585e870a897c26af6b8a29e2c2a8dbd774071d29bfbd4c28ca0a689da01b


### 3)  (2 points) Create a new transaction from address_1 and provide Transaction_ID

In [147]:
# order: tx_id, public key, private key, n_outputs, n_data_outputs
create_new_transaction_3(tx_id_2, address_data_1[1], address_data_1[0], 2, 1) 

Enter previous PKH: 6937728711d918890eeebbbb3819be859f821fe8
Enter Receiver's Address: muXBqnup9A3xScqLohymaX7AxZYz1oAvEB
Enter Value in BTC (0.0) to send:0.0
Enter Data to send: aa
Raw transaction:  01000000011ba09d680aca284cbdbf291d0774d7dba8c2e2298a6baf267c890a875e588c51000000001976a9146937728711d918890eeebbbb3819be859f821fe888acffffffff0200000000000000001976a914999d11d268d59ccc2715b41d6ca5e23b5f4f177d88ac0000000000000000046a0261610000000001000000


Digital Signature:  304402204135bc803bf03a01d4b9f07a7f345af8a724ca5bb970c1d6cb3eae5fbb6ea81602200c1f6802987e1b27ae9661d23b9e7287f5356b99b7ddb14894ccad39d423041e

SigScript(hex):  8a47304402204135bc803bf03a01d4b9f07a7f345af8a724ca5bb970c1d6cb3eae5fbb6ea81602200c1f6802987e1b27ae9661d23b9e7287f5356b99b7ddb14894ccad39d423041e01410447cec9869c73bfbb0d827abd4aca31580ea8ac485664584af6d3fc457f8e7022b44424b5a62111c14372152f4c8457b96a5e2180760d614f09d1d71acde2e664 



Transaction to broadcast:  01000000011ba09d680aca284cbdbf291d0774d7dba8c2e2298a6b

In [101]:
tx_id_3 = 'cda7a0b73840618aafa0e2e931b6fd2d690a7b3d82ba64d655bd51b51d21c262'
pkh_3 = '8fc8aca999cffe5c9dee29a36dc856dd0ccd014e'

print('Test address_2 (from): ', address_2)
print('MultiSignature address (to): ', testnet_address)
print('Previous PKH:' , pkh_3)
print()
print('Transaction ID: ', tx_id_3)

Test address_2 (from):  mtdDKbHP1pYnaSK8EcLn2fb9WdaterTeuX
MultiSignature address (to):  2N239NWhsf6CVW11pdXStVFQkbVb1ojoEek
Previous PKH: 8fc8aca999cffe5c9dee29a36dc856dd0ccd014e

Transaction ID:  cda7a0b73840618aafa0e2e931b6fd2d690a7b3d82ba64d655bd51b51d21c262


### 4) (2 points) From address_2, Send some bitcoins to your 2-of-3 MultiSignature Address.

In [102]:
# order: tx_id, public key, private key, n_outputs, n_data_outputs
create_new_transaction_3(tx_id_3, address_data_2[1], address_data_2[0], 1, 0) 

Enter previous PKH: 8fc8aca999cffe5c9dee29a36dc856dd0ccd014e
Enter Receiver's Address 1: 2N239NWhsf6CVW11pdXStVFQkbVb1ojoEek
Enter Value in BTC (0.0) to send 2N239NWhsf6CVW11pdXStVFQkbVb1ojoEek:0.0
Raw transaction:  010000000162c2211db551bd55d664ba823d7b0a692dfdb631e9e2a0af8a614038b7a0a7cd000000001976a9148fc8aca999cffe5c9dee29a36dc856dd0ccd014e88acffffffff01000000000000000017a91460704961f7a3d8bd71cafb84b7ac900ee0393143870000000001000000


Digital Signature:  3045022100dec027a08c772ed2eca306706ffd89c4b4a01fbcd14df603779fe8f78b9f2fc3022027f3fbf852229aedf1c40d67e37ccbec65e8652b18e5d748f94850c612204665

SigScript(hex):  8b483045022100dec027a08c772ed2eca306706ffd89c4b4a01fbcd14df603779fe8f78b9f2fc3022027f3fbf852229aedf1c40d67e37ccbec65e8652b18e5d748f94850c612204665014104d4241131bbd87e96f216e7492ba4dd6a15b206700bc272ef6862c20935e1c46f115ae53175eef7affb973b6ac857983247dcfa491570412f185a5620daaf33fd 



Transaction to broadcast:  010000000162c2211db551bd55d664ba823d7b0a692dfdb631e9e2a0af8a6140

In [103]:
tx_id_4 = '16e33b7828c3cfe253f0784aa1baad9b3800bc26552630657c2bdf5d68705ab7'

print('MultiSignature address (from): ', testnet_address)
print('Test address_1 (to): ', address_1)
print('Private Key 2: ', address_data_2[0])
print('Private Key 3: ', address_data_3[0])
print('Redeem script: ', rawscript)
print()
print('Transaction ID: ', tx_id_4)

MultiSignature address (from):  2N239NWhsf6CVW11pdXStVFQkbVb1ojoEek
Test address_1 (to):  n1GtCemVB4DpyGuMkwJi1ttkCA9FHN34Eh
Private Key 2:  fdf927024d78f423f09101ab02d6bb09d7eef0a91ccf282489a125360cc40fe6
Private Key 3:  98ad88e9f8cb5b0faaaa48baebd9abb920eca84b02ed008bacd6242bc8c6e7dc
Redeem script:  5241044cb2604bafe6ab1d934bcc414388549350d23dc24cfe064cc1f6f4d47c1c86ee6feb68e9ac8301794b439b827535270ecac9077a5654d4a4283bd0135075e5fe4104d4241131bbd87e96f216e7492ba4dd6a15b206700bc272ef6862c20935e1c46f115ae53175eef7affb973b6ac857983247dcfa491570412f185a5620daaf33fd41042ed8f905c210b9982612cc6fcdc49a4b5933de7b4dd45d10afa788e251529f8634605d4bfd27567337f3041f0107898a2f6bf3b38868a323e4ca8879bd2237d053ae

Transaction ID:  16e33b7828c3cfe253f0784aa1baad9b3800bc26552630657c2bdf5d68705ab7


### 5) (4 points) From 2-of -3 MultiSignature Address. Send some bitcoins back to address_1

In [123]:
create_new_transaction_5()

Enter Reference TxId: 16e33b7828c3cfe253f0784aa1baad9b3800bc26552630657c2bdf5d68705ab7
Enter Previous output index: 0
Enter redeem Script: 5241044cb2604bafe6ab1d934bcc414388549350d23dc24cfe064cc1f6f4d47c1c86ee6feb68e9ac8301794b439b827535270ecac9077a5654d4a4283bd0135075e5fe4104d4241131bbd87e96f216e7492ba4dd6a15b206700bc272ef6862c20935e1c46f115ae53175eef7affb973b6ac857983247dcfa491570412f185a5620daaf33fd41042ed8f905c210b9982612cc6fcdc49a4b5933de7b4dd45d10afa788e251529f8634605d4bfd27567337f3041f0107898a2f6bf3b38868a323e4ca8879bd2237d053ae
Enter Receiver's Address 1: n1GtCemVB4DpyGuMkwJi1ttkCA9FHN34Eh
Enter Value in BTC (0.0) to send n1GtCemVB4DpyGuMkwJi1ttkCA9FHN34Eh:0.0
Enter Private Key: fdf927024d78f423f09101ab02d6bb09d7eef0a91ccf282489a125360cc40fe6
Enter Private Key: 98ad88e9f8cb5b0faaaa48baebd9abb920eca84b02ed008bacd6242bc8c6e7dc
Transaction to broadcast:  0100000001b75a70685ddf2b7c6530265526bc00389badbaa14a78f053e2cfc328783be31600000000fd5e0100483045022100eed73b6758f348d053ccbc774e

In [124]:
tx_id_5 = '988280cb574a2afc1bb74afb39ddba54ba60cb9e905ec9cc8475c7c59857eb0d'
print('Transaction ID final: ', tx_id_5)

Transaction ID final:  988280cb574a2afc1bb74afb39ddba54ba60cb9e905ec9cc8475c7c59857eb0d
